# Aqua Circuit Interoperability Update

_Donny Greenberg, Julien Gacon, Ali Javadi, Steve Wood, 24-Mar-19_

## Basic Vision & End Goal

* Make Aqua use circuits as a first-class currency, and feel more like an algorithms library _next to_ Terra, as users expect, rather than an independent library on top of it
    * No more `construct_circuit()` wrappers in Aqua
* Promote Aqua’s best circuity features into Terra to be broadly useful

## Proposal - Three steps

1. Circuit as a First-class Citizen in Aqua
    1. Aqua algorithms accept circuits directly, no more circuit wrappers
1. Circuit Library with Enhanced QuantumCircuit Families and Convenient Prebuilts
    1. Destination for most of Aqua's enhanced circuit wrappers
    1. Critically, allows for lazily constructed circuit placeholders.
1. Usability Improvements to Promote up to QuantumCircuit
    1. Make circuit construction in Terra more powerful with features in Aqua users like

## 1. Circuit as a First-class Citizen in Aqua

* Anywhere previously calling `construct_circuit` now accepts circuits as-is, no questions asked
* Typehints ask for a circuit, and are indifferent whether a circuit is from the circuit library (below)
* Fully backwards compatible with Aqua's `construct_circuit`-based objects as long as we like
* Maybe warnings where behavior is strange, e.g. no parameters in VQE ansatz

### Demo - VQC with newly built circuits

Below, we demonstrate the execution of the Variational Quantum Classifier using no special circuit construction objects.

In [5]:
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.optimizers import SLSQP

import numpy as np
import itertools

In [2]:
# Learning the one-hot encoding
train_feats = np.eye(3).tolist()
train_labels = [1,2,3]
train = dict(zip(train_labels, train_feats))
print(train)

{1: [1.0, 0.0, 0.0], 2: [0.0, 1.0, 0.0], 3: [0.0, 0.0, 1.0]}


In [3]:
feat_params = ParameterVector('ɸ', length=len(train_feats[0]))
feat_map = QuantumCircuit(3)
depth = 3
for _ in range(depth):
    feat_map.h(qubit=range(3))
    [feat_map.rz(phi=p, qubit=i) for i, p in enumerate(feat_params)]
    [feat_map.crz(theta=p1*p2, control_qubit=q1, target_qubit=q2) 
         for ((q1, p1), (q2,p2)) in itertools.combinations(enumerate(feat_params), 2)]
    feat_map.barrier()
feat_map.draw(fold=1000)

┌───┐┌──────────┐                                                    ░ ┌───┐┌──────────┐                                                    ░ ┌───┐┌──────────┐                                                    ░ 
q_0: |0>┤ H ├┤ Rz(ɸ[0]) ├────────■────────────────■──────────────────────────░─┤ H ├┤ Rz(ɸ[0]) ├────────■────────────────■──────────────────────────░─┤ H ├┤ Rz(ɸ[0]) ├────────■────────────────■──────────────────────────░─
        ├───┤├──────────┤┌───────┴───────┐        │                          ░ ├───┤├──────────┤┌───────┴───────┐        │                          ░ ├───┤├──────────┤┌───────┴───────┐        │                          ░ 
q_1: |0>┤ H ├┤ Rz(ɸ[1]) ├┤ Rz(ɸ[0]*ɸ[1]) ├────────┼────────────────■─────────░─┤ H ├┤ Rz(ɸ[1]) ├┤ Rz(ɸ[0]*ɸ[1]) ├────────┼────────────────■─────────░─┤ H ├┤ Rz(ɸ[1]) ├┤ Rz(ɸ[0]*ɸ[1]) ├────────┼────────────────■─────────░─
        ├───┤├──────────┤└───────────────┘┌───────┴───────┐┌───────┴───────┐ ░ ├───┤├──────────┤└───────────────┘┌───────┴───────┐┌───────┴───────┐ ░ ├───┤├──────────┤└───────────────┘┌───────┴───────┐┌───────┴───────┐ ░ 
q_2: |0>┤ H ├┤ Rz(ɸ[2]) ├─────────────────┤ Rz(ɸ[0]*ɸ[2]) ├┤ Rz(ɸ[1]*ɸ[2]) ├─░─┤ H ├┤ Rz(ɸ[2]) ├─────────────────┤ Rz(ɸ[0]*ɸ[2]) ├┤ Rz(ɸ[1]*ɸ[2]) ├─░─┤ H ├┤ Rz(ɸ[2]) ├─────────────────┤ Rz(ɸ[0]*ɸ[2]) ├┤ Rz(ɸ[1]*ɸ[2]) ├─░─
        └───┘└──────────┘                 └───────────────┘└───────────────┘ ░ └───┘└──────────┘                 └───────────────┘└───────────────┘ ░ └───┘└──────────┘                 └───────────────┘└───────────────┘ ░

In [4]:
# Note: I need to calculate this number
classifier_params = ParameterVector('θ', length=19)
classifier = QuantumCircuit(3)
depth = 3
cp_iter = iter(classifier_params)
next(cp_iter)
for _ in range(depth):
    [classifier.ry(theta=next(cp_iter), qubit=j) for j in classifier.qubits]
    [classifier.crx(theta=next(cp_iter), control_qubit=q1, target_qubit=q2) 
         for (q1, q2) in itertools.combinations(classifier.qubits, 2)]
    classifier.barrier()
classifier.draw(fold=1000)

┌──────────┐                                     ░ ┌──────────┐                                        ░ ┌───────────┐                                        ░ 
q_0: |0>┤ Ry(θ[1]) ├─────■───────────■───────────────────░─┤ Ry(θ[7]) ├──────■────────────■────────────────────░─┤ Ry(θ[13]) ├──────■────────────■────────────────────░─
        ├──────────┤┌────┴─────┐     │                   ░ ├──────────┤┌─────┴─────┐      │                    ░ ├───────────┤┌─────┴─────┐      │                    ░ 
q_1: |0>┤ Ry(θ[2]) ├┤ Rx(θ[4]) ├─────┼───────────■───────░─┤ Ry(θ[8]) ├┤ Rx(θ[10]) ├──────┼────────────■───────░─┤ Ry(θ[14]) ├┤ Rx(θ[16]) ├──────┼────────────■───────░─
        ├──────────┤└──────────┘┌────┴─────┐┌────┴─────┐ ░ ├──────────┤└───────────┘┌─────┴─────┐┌─────┴─────┐ ░ ├───────────┤└───────────┘┌─────┴─────┐┌─────┴─────┐ ░ 
q_2: |0>┤ Ry(θ[3]) ├────────────┤ Rx(θ[5]) ├┤ Rx(θ[6]) ├─░─┤ Ry(θ[9]) ├─────────────┤ Rx(θ[11]) ├┤ Rx(θ[12]) ├─░─┤ Ry(θ[15]) ├─────────────┤ Rx(θ[17]) ├┤ Rx(θ[18]) ├─░─
        └──────────┘            └──────────┘└──────────┘ ░ └──────────┘             └───────────┘└───────────┘ ░ └───────────┘             └───────────┘└───────────┘ ░

In [ ]:
vqc = VQC(optimizer=SLSQP(), feature_map=feat_map, var_form=classifier, training_dataset=train, test_dataset=train)
vqc.run()

## 2. Circuit Library with Enhanced Flexibility Circuits and Convenient Prebuilts

_Proposal: Move Aqua's circuit-constructor objects - e.g. Ansatze, QFTs, Arithmetic - into a broadly useful circuit-library as flexible QuantumCircuit objects with enhanced features._

#### New Concepts in the Circuit Library
* Circuit Blueprints: Enhanced QuantumCircuit objects which are lazily populated and constructed, but print and interact as bona-fide circuits.
    * Not a new class, simply subclasses of QuantumCircuit which match the QuantumCircuit interface
    * Users generally shouldn't notice the difference, unless digging into circuit guts in debugging
    * Properties such as `.data`, `.parameters`, etc. which require real circuits, trigger construction and caching of constructed circuit
    * Meta-parameters, such as ansatz depth or connectivity, are mutable and edited lightly due to lazy construction. Setters trigger cached circuit wipe
* Circuit Families
    * Collections of circuit blueprints or prebuilt circuits with extensions or use-case specific features - e.g. `PermutationCircuit`s can include properties which `ArithmeticCircuit`s do not.
    * Allow for more aggressive convenience functionality for specific use cases e.g. Ansatz automatically allocating parameters during construction.

#### Options for placement
1.   Inside Terra, with integration tests
1.   Inside Aqua, with Qiskit-wide utility, no Aqua concepts
1.   In its own repo

#### Options for circuit organization
1. By purpose - Organization followed by Aqua
    1. data_preparation
        1. feature_maps
        1. probability_distributions
    1. Ansatze
        1. TwoLocalCircuit
        1. NLocalCircuit
        1. Ry
        1. RyRz
        1. SwapRz
        1. UCCSD? QAOA? TASP? (would be complicated without certain Operator work, or restricted to one simulation algorithm with Paulis only).
    1. Arithmetic
        1. Adders
        1. Reciprocal
    1. basis_changes
        1. QFT
        1. QWT
        1. DCT
        1. Pauli CoB
    1. Oracle
        1. Truth table
        1. Logical expression
        1. Phase oracle
        1. Permutation oracle
1. By form factor - Organization followed by internal quantum-circuits repo
    1. Random uniform
    1. Hardware efficient
    1. Near-Clifford / graph states
    1. Quantum volume
    1. Quantum Fourier transform
    1. Ideal HLF circuits
    1. Hamming weight
    1. Hidden shift with bent Boolean functions
    1. Multiply-controlled NOT gate
    1. IQP circuits
    1. Fourier checking
1.   Case-by-case
    1. Some families are specified by their purpose to be able to add functionality and methods, while other are specified by form factor to meet user expectations. 
    1. Circuits can act as placeholders (e.g. permutation) to be filled in by a choice of several form factors later. 
    1. Circuits can also import circuits from other families so both practical and theoretical expectations are met without code duplication.
    1. TODO merged structure
    
_Proposal: HardwareEfficient base class - abstract `target_backend` setter (QFTs, ansatze, etc. can choose how to interpret what to do with hardware and provide specially tailored version)_

### Demo 2 - Powerful New QuantumCircuit Objects

In [8]:
from chemistry.code.molecule import Molecule
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.aqua.algorithms import VQE
# from qiskit.circuit_.library import RyRz

from qiskit import BasicAer
qasm = BasicAer.get_backend('qasm_simulator')

In [ ]:
hh = Molecule(geometry=[['H', [0., 0., 1.]],
                       ['H', [0., 0.45, 1.]],
                       ])
hamiltonian = hh.get_qubitop_hamiltonian()
molecular_wf = HartreeFock(hh) + RyRz(depth=3, entanglement='linear')
gse = VQE(var_form=molecular_wf, optimizer=SLSQP, backend=qasm).compute_minimum_eigenvalue(hamiltonian)
print(gse)

In [ ]:
molecular_qf.target_backend = IBMQ.get_backend('ibmq_valencia')

## 3. QuantumCircuit Usability Improvements

Aqua's circuit_constructors have accumulated many powerful features not present in QuantumCircuit. No changes are strictly needed to QuantumCircuit to support the above proposals, but we can promote some of these improvements up to QuantumCircuit base to make these features broadly available.

*   Suggested for immediate promotion:
    *   Mutable qubit number (delete qubits, or extend circuit) for anonymous register circuits
    *   .parameters returns a list instead of a set
*   Further opportunities for radical circuit control
    *   Lazy parameterization
    *   Lazy broadcasting
    *   What we’d need to do to implement these two

Demo 4 - Interface demo of further opportunities

In [ ]:
# Working notes - Captures new features but doesn't highlight them exactly
my_c = QuantumCircuit(4)
my_c.h(qubits='all')
my_c.cu3(qubits='full', theta=.5) # other two parameters are set to be parameterized under the hood
my_c.h(qubits='all')
my_c.rz(qubits='all', phi=Parameter('theta')) # Sets the same parameter for all of them